# Testing

In [23]:
import pandas as pd
from fine_tune_inference import FineTuneInference

In [3]:
def clean_messages(message: list):
    input_message = []
    gt_answer = None
    for m in message:
        if m['role'] == 'user' or m['role'] == 'system':
            input_message.append(m)
        elif m['role'] == 'assistant':
            gt_answer = m['content']
    return input_message, gt_answer

In [32]:
val_df = pd.read_json('Sample-Select/val.jsonl', lines=True, orient='records')
train_df = pd.read_json('Sample-Select/train.jsonl', lines=True, orient='records')

In [4]:
fti = FineTuneInference(adapter_dir='Sample-Size-LLM/sample-size-sft-lora')

Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

ValueError: Can't find 'adapter_config.json' at 'Sample-Size-LLM/sample-size-sft-lora'

In [6]:
val_df.messages[1]

[{'role': 'system',
  'content': 'You are an expert data analyst. Given summarized data from a tabular dataset, you will be asked to perform various statistical analyses. Return ONLY the final sample size as an integer.'},
 {'role': 'user',
  'content': 'Population size: 107\nConfidence level: 0.9\nTolerable error rate: 0.1\nAssumed probability of success: 0.5\nRounding: ceil\nUsed FPC: True\n\nReturn ONLY the final sample size.'},
 {'role': 'assistant', 'content': '42'}]

In [5]:
test_messages = []
actual = []
for val in val_df.messages:
    imess, gt = clean_messages(val)
    test_messages.append(imess)
    actual.append(gt)

In [9]:
predictions = [fti.predict(x) for x in test_messages]

In [10]:
pd.DataFrame({
    'actual': actual,
    'predictions': predictions
})

,actual,predictions
0,49,49
1,42,42
2,36,36
3,40,40
4,35,34
5,49,48
6,47,48
7,47,47
8,35,34
9,33,32


In [7]:
actual[0]

'|   Variance |\n|-----------:|\n|         -6 |\n|         10 |\n|        -19 |\n|        -21 |\n|        -19 |'

In [25]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct", use_fast=True)


In [34]:
max([len(tokenizer.encode(str(t))) for t in train_df.messages])

2521

In [8]:

new_task_message = [
    {
        "role": "system",
        "content": "You are an expert data analyst. Given summarized data from a tabular dataset, you will be asked to perform various statistical analyses. Return ONLY the quarter-on-quarter variance as a floating point number."},
    {
        "role": "user",
        "content": f"Quarterly Data: {qdf}\n\nReturn ONLY the quarter-on-quarter variance."}
]

In [9]:
fti.predict(new_task_message, 1000)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'1.51'

In [12]:
train_df.messages[0]

[{'role': 'system',
  'content': 'You are an expert data analyst. Given summarized data from a tabular dataset, you will be asked to perform various statistical analyses. Return ONLY the quarter-on-quarter variances for each record as a rounded integer.'},
 {'role': 'user',
  'content': 'Quarterly Data: |   Q2 |   Q3 |\n|-----:|-----:|\n|   58 |   61 |\n|   23 |   30 |\n|   84 |   85 |\n|   37 |   43 |\n|   71 |   70 |\n\n\nReturn ONLY the quarter-on-quarter variances for each record as a rounded integer.'},
 {'role': 'assistant',
  'content': '|   Variance |\n|-----------:|\n|         -5 |\n|        -23 |\n|         -1 |\n|        -14 |\n|          1 |'}]

In [ ]:
# There is data in the attached file, <FILE>. I need to use the data to be able to accomplish <TASK> where the end result is <GT EXAMPLE>. Following the format of the data in the file, generate 400 synthetic examples of both the data in the file and a corresponding ground truth result.

In [33]:
sample_size_calculation1 = {
  "confidence_level": 0.9,
  "z_value": 1.64485,
  "tolerable_error": 0.1,
  "assumed_p": 0.5,
  "n0_unbounded": 67.63828806249998,
  "population_size": 203,
  "finite_population_corrected_n": 50.92219126352285,
  "final_sample_size": 51,
  "used_fpc": True,
  "rounding": "ceil",
  "formula_notes": {
    "n0": "(Z^2 * p * (1-p)) / E^2",
    "fpc": "(N * n0) / (N + n0 - 1)"
  }
}

In [35]:
population_size = sample_size_calculation1['population_size']
confidence_level = sample_size_calculation1['confidence_level']
tolerable_error = sample_size_calculation1['tolerable_error']
assumed_p = sample_size_calculation1['assumed_p']
ground_truth = sample_size_calculation1['final_sample_size']

In [36]:
sample_training_data = {
    'prompt': [
        {'content': "You are an expert data analyst. Given a tabular set of data, you will be asked to perform various statistical analyses.",
         'role': 'system'},
        {'content': f"There is a data set that has a population size of {population_size}. Calculate the required sample size for audit testing using the following statistical metrics:\n\n- Confidence level: {confidence_level}\n- Tolerable error rate: {tolerable_error}\n- Assumed probability of success: {assumed_p}\n\nReturn ONLY the final sample size.",
         'role': 'user'}
    ],
    'ground_truth': f"Final sample size: {ground_truth}"
}

In [5]:
import pandas as pd
import numpy as np

In [8]:
test_df = pd.DataFrame({
    'randname': ['a', 'b', 'c', 'd'],
    'digits': [1, 2, 3, np.nan],
    'str_digits': ['1', '2', '3', '']
})

In [11]:
col = test_df.str_digits

col1 = pd.to_numeric(col, errors="coerce")
col1 = col1.fillna(0)
col1

0    1.0
1    2.0
2    3.0
3    0.0
Name: str_digits, dtype: float64

In [12]:
test_df.shape[0]

4

In [13]:
rng = np.random.default_rng()


In [22]:
np.random.randint(25,100)

91